In [1]:
import numpy as np
import matplotlib.pyplot as plt

import sensors as sn
import em_tools as em
import subspace_methods as sm

In [2]:
wave_length = 1.5
inter_sensor_dist = 0.75
DIST_RATIO = inter_sensor_dist / wave_length

Num_sensors = 1000
Num_emitters = 1
sample_size = 50

failing_sensors = np.arange(500, 1000)
gap_ratio = 0.5 * np.ones_like(failing_sensors, dtype=np.float32)

theta_rad = np.array([0.7]) # Угловые координаты источников (DoA) в радианах
theta_deg = np.rad2deg(theta_rad) # Угловые координаты источников (DoA) в градусах

signal_pressure = 0.01
noise_pressure = 0.0016

# Расстояния между источником и антенной решеткой, два случая
emitter_receiver_distance1 = 100
emitter_receiver_distance2 = 1000

P1 = (sn.signal_variance(signal_pressure, emitter_receiver_distance1) * 
      np.eye(Num_emitters, dtype=np.float64)) # Ковариация сигналов для ближнего источника
P2 = (sn.signal_variance(signal_pressure, emitter_receiver_distance2) * 
      np.eye(Num_emitters, dtype=np.float64)) # Ковариация сигналов для дальнего источника

Q = sn.noise_variance(signal_pressure) * np.eye(Num_sensors, dtype=np.float64) # Ковариация шумов

A = (np.exp(-2j * np.pi * DIST_RATIO * np.arange(Num_sensors).reshape(-1,1) * 
             np.sin(theta_rad))) # Матрица векторов направленности

# Генерация сигналов, шумов и наблюдений
N = sn.gss(Num_sensors, sample_size, Q)

S1 = sn.gss(Num_emitters, sample_size, P1)
S2 = sn.gss(Num_emitters, sample_size, P2)

X1 = (A @ S1.T + N.T).T
X2 = (A @ S2.T + N.T).T

X1_with_mv = sn.MCAR(X1, failing_sensors, gap_ratio)
X2_with_mv = sn.MCAR(X2, failing_sensors, gap_ratio)


In [3]:
sm.trunc_MUSIC(X1_with_mv, Num_emitters)

array([0.29670597])

In [4]:
sm.trunc_ESPRIT(X1_with_mv, Num_emitters)

array([-0.91864917])

In [5]:
sm.mean_imput_MUSIC(X1_with_mv, Num_emitters)

array([0.65449847])

In [6]:
sm.mean_imput_ESPRIT(X1_with_mv, Num_emitters)

array([1.10266734])